In [ ]:
import pandas as pd
import json
import os

# Create sample data if you don't have the actual CSV files
# This mimics what would be in your CSV files based on the code you provided
def create_sample_data():
    # Socioeconomic groups data
    socio_data = {
        'Socioøkonomisk status': [
            "Gennemsnitshusstand", "Selvstændig", "Lønmodtager på højeste niveau", 
            "Lønmodtager på mellemniveau", "Lønmodtager på grundniveau", 
            "Arbejdsløs", "Uddannelsessøgende", "Pensionist, efterlønsmodtager", 
            "Ude af erhverv i øvrigt"
        ],
        '09.8 Pakkerejser': [4710, 4581, 7628, 6538, 4413, 155, 1673, 4393, 1320],
        '11.1 Restaurationstjenester': [14197, 18340, 24289, 19945, 16423, 16515, 11459, 7323, 7367],
        '11.2 Overnatningsfaciliteter': [5138, 10129, 9572, 5450, 5677, 11420, 2294, 2972, 2068]
    }
    
    # Age groups data - create sample data
    age_data = {
        'Age Group': ['Under 30', '30-39', '40-49', '50-59', '60-69', 'Over 70'],
        'Package Holidays - Fixed Prices': [1500, 3500, 5500, 6000, 7000, 4000],
        'Restaurant Services - Fixed Prices': [12000, 15000, 18000, 17000, 14000, 8000],
        'Accommodation Services - Fixed Prices': [3000, 4500, 6000, 5500, 4000, 2500]
    }
    
    # Region data - create sample data
    region_data = {
        'Region': ['Hovedstaden', 'Sjælland', 'Syddanmark', 'Midtjylland', 'Nordjylland'],
        'Package Holidays - Fixed Prices': [5500, 4000, 3800, 4200, 3500],
        'Restaurant Services - Fixed Prices': [18000, 13000, 12000, 13500, 11000],
        'Accommodation Services - Fixed Prices': [6000, 4500, 4000, 4500, 3800]
    }
    
    # Create DataFrames
    socioeconomic = pd.DataFrame(socio_data)
    consumption = pd.DataFrame(age_data)
    region = pd.DataFrame(region_data)
    
    return socioeconomic, consumption, region

# Either read from CSV files or use sample data
try:
    socioeconomic = pd.read_csv("../data/socioeconomic.csv")
    consumption = pd.read_csv("../data/Consumption.csv")
    region = pd.read_csv("../data/Region.csv")
    print("Successfully loaded CSV files")
except Exception as e:
    print(f"Using sample data: {e}")
    socioeconomic, consumption, region = create_sample_data()

# Translations for socioeconomic groups
socio_translations = {
    "Gennemsnitshusstand": "Average Household",
    "Selvstændig": "Self-employed",
    "Lønmodtager på højeste niveau": "High Income",
    "Lønmodtager på mellemniveau": "Medium Income",
    "Lønmodtager på grundniveau": "Basic Income",
    "Arbejdsløs": "Unemployed", 
    "Uddannelsessøgende": "Student",
    "Pensionist, efterlønsmodtager": "Pensioner",
    "Ude af erhverv i øvrigt": "Not in Workforce"
}

# Calculate season-based travel data (simulated)
# This adds an additional dimension to the visualization
season_data = {
    'Season': ['Winter', 'Spring', 'Summer', 'Fall'],
    'Groups': {
        'High Income': [30, 25, 25, 20],        # % distribution
        'Medium Income': [20, 25, 35, 20],
        'Basic Income': [15, 20, 45, 20],
        'Student': [25, 15, 45, 15],
        'Pensioner': [30, 25, 25, 20]
    }
}

# Clean and prepare socioeconomic data
socio_df = socioeconomic.copy()
socio_df.columns = [col.strip() for col in socio_df.columns]

socio_df = socio_df.rename(columns={
    'Socioøkonomisk status': 'Group',
    '09.8 Pakkerejser': 'Travel Packages',
    '11.1 Restaurationstjenester': 'Restaurants',
    '11.2 Overnatningsfaciliteter': 'Accommodation'
})

# Add English translations
socio_df['Group_EN'] = socio_df['Group'].map(socio_translations)

# Add total spending column
socio_df['Total'] = socio_df['Travel Packages'] + socio_df['Restaurants'] + socio_df['Accommodation']

# Calculate percentages for each category
socio_df['Package_Pct'] = (socio_df['Travel Packages'] / socio_df['Total'] * 100).round(1)
socio_df['Restaurant_Pct'] = (socio_df['Restaurants'] / socio_df['Total'] * 100).round(1)
socio_df['Accommodation_Pct'] = (socio_df['Accommodation'] / socio_df['Total'] * 100).round(1)

# Prepare data for all datasets
datasets = {
    'socioeconomic': {
        'title': 'Travel Budget by Socioeconomic Group',
        'data': []
    },
    'season': {
        'title': 'Travel Seasons by Demographic Group',
        'data': []
    }
}

# Add age data if available
if 'consumption' in locals():
    if 'Age Group' in consumption.columns:
        # Find or use the appropriate columns
        try:
            package_col = [col for col in consumption.columns if 'Package Holidays - Fixed Prices' in col][0]
            restaurant_col = [col for col in consumption.columns if 'Restaurant Services - Fixed Prices' in col][0]
            accommodation_col = [col for col in consumption.columns if 'Accommodation Services - Fixed Prices' in col][0]
        except IndexError:
            package_col = 'Package Holidays - Fixed Prices'
            restaurant_col = 'Restaurant Services - Fixed Prices'
            accommodation_col = 'Accommodation Services - Fixed Prices'
        
        # Prepare age data
        datasets['age'] = {
            'title': 'Travel Budget by Age Group',
            'data': []
        }
        
        for idx, row in consumption.iterrows():
            total = row[package_col] + row[restaurant_col] + row[accommodation_col]
            datasets['age']['data'].append({
                'group': row['Age Group'],
                'translated': row['Age Group'],  # No translation needed
                'Rejsepakker': row[package_col],
                'Restauranter': row[restaurant_col],
                'Overnatning': row[accommodation_col],
                'Total': total,
                'Package_Pct': round(row[package_col] / total * 100, 1),
                'Restaurant_Pct': round(row[restaurant_col] / total * 100, 1),
                'Accommodation_Pct': round(row[accommodation_col] / total * 100, 1)
            })

# Add region data if available
if 'region' in locals():
    if 'Region' in region.columns:
        # Find or use the appropriate columns
        try:
            package_col = [col for col in region.columns if 'Package Holidays - Fixed Prices' in col][0]
            restaurant_col = [col for col in region.columns if 'Restaurant Services - Fixed Prices' in col][0]
            accommodation_col = [col for col in region.columns if 'Accommodation Services - Fixed Prices' in col][0]
        except IndexError:
            package_col = 'Package Holidays - Fixed Prices'
            restaurant_col = 'Restaurant Services - Fixed Prices'
            accommodation_col = 'Accommodation Services - Fixed Prices'
        
        # Prepare region data
        datasets['region'] = {
            'title': 'Travel Budget by Region',
            'data': []
        }
        
        for idx, row in region.iterrows():
            total = row[package_col] + row[restaurant_col] + row[accommodation_col]
            datasets['region']['data'].append({
                'group': row['Region'],
                'translated': row['Region'],  # No translation needed
                'Rejsepakker': row[package_col],
                'Restauranter': row[restaurant_col],
                'Overnatning': row[accommodation_col],
                'Total': total,
                'Package_Pct': round(row[package_col] / total * 100, 1),
                'Restaurant_Pct': round(row[restaurant_col] / total * 100, 1),
                'Accommodation_Pct': round(row[accommodation_col] / total * 100, 1)
            })

# Prepare socioeconomic data
for idx, row in socio_df.iterrows():
    datasets['socioeconomic']['data'].append({
        'group': row['Group'],
        'translated': row['Group_EN'],
        'Rejsepakker': row['Travel Packages'],
        'Restauranter': row['Restaurants'],
        'Overnatning': row['Accommodation'],
        'Total': row['Total'],
        'Package_Pct': row['Package_Pct'],
        'Restaurant_Pct': row['Restaurant_Pct'],
        'Accommodation_Pct': row['Accommodation_Pct']
    })

# Prepare seasonal data
for season in season_data['Season']:
    season_idx = season_data['Season'].index(season)
    datasets['season']['data'].append({
        'season': season,
        'HighIncome': season_data['Groups']['High Income'][season_idx],
        'MediumIncome': season_data['Groups']['Medium Income'][season_idx],
        'BasicIncome': season_data['Groups']['Basic Income'][season_idx],
        'Student': season_data['Groups']['Student'][season_idx],
        'Pensioner': season_data['Groups']['Pensioner'][season_idx]
    })

# Convert data to JSON for the React component
json_data = json.dumps(datasets)

# Create enhanced React component with Recharts that includes multiple visualization types
html_template = f'''
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Danish Travel Budget Visualizations</title>
  
  <!-- React and ReactDOM -->
  <script src="https://unpkg.com/react@17/umd/react.production.min.js" crossorigin></script>
  <script src="https://unpkg.com/react-dom@17/umd/react-dom.production.min.js" crossorigin></script>
  
  <!-- Recharts for visualizations -->
  <script src="https://unpkg.com/recharts@2.1.12/umd/Recharts.min.js" crossorigin></script>
  
  <!-- Babel for JSX -->
  <script src="https://unpkg.com/@babel/standalone/babel.min.js"></script>
  
  <style>
    body {{
      font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
      margin: 0;
      padding: 20px;
      background-color: #f7f9fc;
      color: #333;
    }}
    
    .container {{
      max-width: 1100px;
      margin: 0 auto;
    }}
    
    .header {{
      text-align: center;
      margin-bottom: 30px;
    }}
    
    .header h1 {{
      font-size: 32px;
      color: #2c3e50;
      margin-bottom: 8px;
    }}
    
    .header p {{
      font-size: 16px;
      color: #7f8c8d;
      max-width: 700px;
      margin: 0 auto;
    }}
    
    .chart-container {{
      background-color: white;
      padding: 25px;
      margin-bottom: 40px;
      border-radius: 12px;
      box-shadow: 0 4px 12px rgba(0,0,0,0.05);
      transition: all 0.3s ease;
    }}
    
    .chart-container:hover {{
      box-shadow: 0 6px 16px rgba(0,0,0,0.1);
    }}
    
    .chart-title {{
      text-align: center;
      margin-bottom: 25px;
      color: #2c3e50;
      font-size: 22px;
      font-weight: 600;
    }}
    
    .chart-subtitle {{
      text-align: center;
      color: #7f8c8d;
      font-size: 15px;
      margin-bottom: 25px;
    }}
    
    .tabs {{
      display: flex;
      justify-content: center;
      margin-bottom: 30px;
      flex-wrap: wrap;
    }}
    
    .tab-button {{
      background: none;
      border: 2px solid #e0e6ed;
      padding: 10px 20px;
      margin: 5px;
      cursor: pointer;
      border-radius: 25px;
      font-size: 14px;
      color: #7f8c8d;
      font-weight: 500;
      transition: all 0.2s ease;
    }}
    
    .tab-button:hover {{
      background-color: #f5f7fa;
      color: #34495e;
    }}
    
    .tab-button.active {{
      background-color: #3498db;
      color: white;
      border-color: #3498db;
      font-weight: 600;
    }}
    
    .tooltip {{
      background-color: white;
      padding: 12px 18px;
      border-radius: 8px;
      box-shadow: 0 4px 12px rgba(0,0,0,0.15);
      border: 1px solid #eaeaea;
    }}
    
    .tooltip-title {{
      font-weight: 600;
      margin-bottom: 8px;
      color: #2c3e50;
      border-bottom: 1px solid #eee;
      padding-bottom: 5px;
    }}
    
    .tooltip-content {{
      font-size: 14px;
      color: #34495e;
    }}
    
    .tooltip-label-blue {{
      color: #3498db;
    }}
    
    .tooltip-label-green {{
      color: #2ecc71;
    }}
    
    .tooltip-label-orange {{
      color: #e67e22;
    }}
    
    .tooltip-total {{
      margin-top: 8px;
      font-weight: 600;
      border-top: 1px solid #eee;
      padding-top: 8px;
      color: #2c3e50;
    }}
    
    .hint {{
      text-align: center;
      font-size: 14px;
      color: #95a5a6;
      margin-top: 15px;
      background-color: #f8f9fa;
      padding: 8px 15px;
      border-radius: 6px;
    }}
    
    .controls-row {{
      display: flex;
      justify-content: space-between;
      align-items: center;
      margin-bottom: 20px;
      flex-wrap: wrap;
    }}
    
    .sort-control {{
      display: flex;
      align-items: center;
      background-color: #f8f9fa;
      padding: 5px 12px;
      border-radius: 20px;
    }}
    
    .sort-control label {{
      margin-right: 8px;
      font-size: 14px;
      color: #7f8c8d;
    }}
    
    .sort-control select {{
      padding: 6px 10px;
      border-radius: 4px;
      border: 1px solid #ddd;
      background-color: white;
      font-size: 14px;
      color: #34495e;
      outline: none;
    }}
    
    .sort-control select:focus {{
      border-color: #3498db;
    }}
    
    .highlight-hint {{
      font-size: 13px;
      color: #95a5a6;
    }}
    
    .view-toggle {{
      display: flex;
      align-items: center;
      margin-bottom: 20px;
    }}
    
    .view-toggle-button {{
      background: none;
      border: 1px solid #ddd;
      padding: 8px 15px;
      margin-right: 6px;
      cursor: pointer;
      font-size: 14px;
      border-radius: 6px;
      color: #7f8c8d;
      transition: all 0.2s ease;
    }}
    
    .view-toggle-button:hover {{
      background-color: #f5f7fa;
    }}
    
    .view-toggle-button.active {{
      background-color: #3498db;
      color: white;
      border-color: #3498db;
    }}
    
    .insights {{
      background-color: #f8f9fa;
      padding: 20px 25px;
      margin-top: 25px;
      border-radius: 8px;
      border-left: 4px solid #3498db;
    }}
    
    .insights h3 {{
      font-size: 18px;
      color: #2c3e50;
      margin-top: 0;
      margin-bottom: 15px;
    }}
    
    .insights ul {{
      margin: 0;
      padding-left: 20px;
    }}
    
    .insights li {{
      margin-bottom: 10px;
      line-height: 1.5;
      color: #34495e;
    }}
    
    .insights li strong {{
      color: #2c3e50;
    }}
    
    .seasonal-grid {{
      display: grid;
      grid-template-columns: repeat(auto-fill, minmax(200px, 1fr));
      gap: 20px;
      margin-top: 30px;
    }}
    
    .seasonal-card {{
      background-color: white;
      border-radius: 10px;
      padding: 20px;
      box-shadow: 0 2px 8px rgba(0,0,0,0.05);
      transition: all 0.3s ease;
    }}
    
    .seasonal-card:hover {{
      transform: translateY(-5px);
      box-shadow: 0 8px 16px rgba(0,0,0,0.1);
    }}
    
    .seasonal-card h3 {{
      font-size: 18px;
      margin-top: 0;
      margin-bottom: 15px;
      color: #2c3e50;
      text-align: center;
    }}
    
    .seasonal-value {{
      font-size: 15px;
      margin: 8px 0;
      display: flex;
      justify-content: space-between;
      align-items: center;
    }}
    
    .seasonal-value-bar {{
      height: 8px;
      background-color: #ecf0f1;
      border-radius: 10px;
      margin-top: 4px;
      overflow: hidden;
    }}
    
    .seasonal-value-fill {{
      height: 100%;
      border-radius: 10px;
    }}
    
    @media (max-width: 768px) {{
      .chart-container {{
        padding: 20px 15px;
      }}
      
      .tabs {{
        padding: 0 10px;
      }}
      
      .tab-button {{
        padding: 8px 15px;
        font-size: 13px;
      }}
      
      .controls-row {{
        flex-direction: column;
        align-items: flex-start;
      }}
      
      .sort-control {{
        margin-top: 12px;
        width: 100%;
      }}
      
      .sort-control select {{
        flex-grow: 1;
      }}
      
      .seasonal-grid {{
        grid-template-columns: 1fr;
      }}
    }}
  </style>
</head>
<body>
  <div id="root"></div>

  <script type="text/babel">
    // Parse the data
    const datasets = {json_data};
    
    // Destructure Recharts components
    const {{
      RadarChart, 
      PolarGrid, 
      PolarAngleAxis, 
      PolarRadiusAxis, 
      Radar, 
      Legend, 
      Tooltip, 
      ResponsiveContainer,
      BarChart,
      Bar,
      XAxis,
      YAxis,
      CartesianGrid,
      PieChart,
      Pie,
      Cell,
      Sector,
      AreaChart,
      Area,
      ComposedChart,
      Line
    }} = Recharts;
    
    // Custom tooltip component with percentages and totals
    const CustomTooltip = ({{ active, payload, label }}) => {{
      if (active && payload && payload.length) {{
        const data = payload[0].payload;
        
        if (data.Total) {{
          // For spending data
          return (
            <div className="tooltip">
              <div className="tooltip-title">{{data.translated || label}}</div>
              <div className="tooltip-content">
                <div className="tooltip-label-blue">
                  Rejsepakker: {{data.Rejsepakker?.toLocaleString()}} DKK ({{data.Package_Pct}}%)
                </div>
                <div className="tooltip-label-green">
                  Restauranter: {{data.Restauranter?.toLocaleString()}} DKK ({{data.Restaurant_Pct}}%)
                </div>
                <div className="tooltip-label-orange">
                  Overnatning: {{data.Overnatning?.toLocaleString()}} DKK ({{data.Accommodation_Pct}}%)
                </div>
                <div className="tooltip-total">
                  Total: {{data.Total?.toLocaleString()}} DKK
                </div>
              </div>
            </div>
          );
        }} else if (data.season) {{
          // For seasonal data
          return (
            <div className="tooltip">
              <div className="tooltip-title">{{data.season}} Season</div>
              <div className="tooltip-content">
                <div>High Income: {{data.HighIncome}}%</div>
                <div>Medium Income: {{data.MediumIncome}}%</div>
                <div>Basic Income: {{data.BasicIncome}}%</div>
                <div>Student: {{data.Student}}%</div>
                <div>Pensioner: {{data.Pensioner}}%</div>
              </div>
            </div>
          );
        }}
      }}
      return null;
    }};
    
    // Custom component for active shape in pie chart
    const renderActiveShape = (props) => {{
      const RADIAN = Math.PI / 180;
      const {{ cx, cy, midAngle, innerRadius, outerRadius, startAngle, endAngle,
        fill, payload, percent, value }} = props;
      const sin = Math.sin(-RADIAN * midAngle);
      const cos = Math.cos(-RADIAN * midAngle);
      const sx = cx + (outerRadius + 10) * cos;
      const sy = cy + (outerRadius + 10) * sin;
      const mx = cx + (outerRadius + 30) * cos;
      const my = cy + (outerRadius + 30) * sin;
      const ex = mx + (cos >= 0 ? 1 : -1) * 22;
      const ey = my;
      const textAnchor = cos >= 0 ? 'start' : 'end';
    
      return (
        <g>
          <text x={{cx}} y={{cy}} dy={{8}} textAnchor="middle" fill={{fill}} fontSize={{16}} fontWeight={600}>
            {{payload.translated}}
          </text>
          <Sector
            cx={{cx}}
            cy={{cy}}
            innerRadius={{innerRadius}}
            outerRadius={{outerRadius}}
            startAngle={{startAngle}}
            endAngle={{endAngle}}
            fill={{fill}}
          />
          <Sector
            cx={{cx}}
            cy={{cy}}
            startAngle={{startAngle}}
            endAngle={{endAngle}}
            innerRadius={{outerRadius + 6}}
            outerRadius={{outerRadius + 10}}
            fill={{fill}}
          />
          <path d={{`M${{sx}},${{sy}}L${{mx}},${{my}}L${{ex}},${{ey}}`}} stroke={{fill}} fill="none" />
          <circle cx={{ex}} cy={{ey}} r={{2}} fill={{fill}} stroke="none" />
          <text x={{ex + (cos >= 0 ? 1 : -1) * 12}} y={{ey}} textAnchor={{textAnchor}} fill="#333" fontSize={{14}}>
            {{`${{value.toLocaleString()}} DKK`}}
          </text>
          <text x={{ex + (cos >= 0 ? 1 : -1) * 12}} y={{ey}} dy={{18}} textAnchor={{textAnchor}} fill="#999" fontSize={{12}}>
            {{`(${{(percent * 100).toFixed(1)}}%)`}}
          </text>
        </g>
      );
    }};
    
    // Custom Active Dot for Line Charts
    const CustomizedActiveDot = (props) => {{
      const {{ cx, cy, stroke, payload, value }} = props;
    
      return (
        <g>
          <circle cx={{cx}} cy={{cy}} r={{8}} stroke={{stroke}} strokeWidth={{2}} fill="#FFF" />
          <circle cx={{cx}} cy={{cy}} r={{4}} stroke="none" fill={{stroke}} />
        </g>
      );
    }};
    
    const App = () => {{
      // State to track which dataset is active
      const [activeDataset, setActiveDataset] = React.useState('socioeconomic');
      
      // State to track chart view
      const [chartView, setChartView] = React.useState('radar');
      
      // State to track highlighted category
      const [activeCategory, setActiveCategory] = React.useState(null);
      
      // State for sorting
      const [sortBy, setSortBy] = React.useState('default');
      
      // State for active pie slice
      const [activePieIndex, setActivePieIndex] = React.useState(0);
      
      // Get available datasets (excluding seasonal for now as it has different structure)
      const availableDatasets = Object.keys(datasets).filter(key => key !== 'season');
      
      // Format display names
      const displayNames = {{
        'socioeconomic': 'Socioeconomic Groups',
        'age': 'Age Groups',
        'region': 'Regions',
        'season': 'Travel Seasons'
      }};
      
      // Chart view options
      const viewOptions = [
        {{ id: 'radar', name: 'Radar Chart' }},
        {{ id: 'bar', name: 'Bar Chart' }},
        {{ id: 'pie', name: 'Pie Chart' }},
        {{ id: 'composed', name: 'Advanced' }}
      ];
      
      // Sort the data based on the sort selection
      const sortedData = React.useMemo(() => {{
        if (!datasets[activeDataset]) return [];
        
        if (sortBy === 'default') return datasets[activeDataset].data;
        
        return [...datasets[activeDataset].data].sort((a, b) => {{
          if (sortBy === 'total') {{
            return b.Total - a.Total; // Descending
          }}
          return b[sortBy] - a[sortBy]; // Descending by category
        }});
      }}, [activeDataset, sortBy, datasets]);
      
      // Handle legend click to toggle category highlight
      const handleLegendClick = (entry) => {{
        if (activeCategory === entry.dataKey) {{
          setActiveCategory(null); // Deselect if already selected
        }} else {{
          setActiveCategory(entry.dataKey); // Select the clicked category
        }}
      }};
      
      // Handle pie chart mouse enter
      const handlePieEnter = (_, index) => {{
        setActivePieIndex(index);
      }};
      
      // Colors for the charts
      const colors = {{
        Rejsepakker: '#3498db',  // Blue
        Restauranter: '#2ecc71', // Green
        Overnatning: '#e67e22',  // Orange
        Total: '#9b59b6'         // Purple
      }};
      
      // Colors for pie chart
      const COLORS = ['#3498db', '#2ecc71', '#e67e22', '#f1c40f', '#e74c3c', '#9b59b6', '#1abc9c', '#34495e', '#d35400'];
      
      // Calculate spending metrics for insights
      const calculateInsights = () => {{
        if (!sortedData || sortedData.length === 0) return null;
        
        // Find highest and lowest total spenders
        const highestSpender = [...sortedData].sort((a, b) => b.Total - a.Total)[0];
        const lowestSpender = [...sortedData].sort((a, b) => a.Total - b.Total)[0];
        
        // Find who spends most on each category
        const highestPackage = [...sortedData].sort((a, b) => b.Rejsepakker - a.Rejsepakker)[0];
        const highestRestaurant = [...sortedData].sort((a, b) => b.Restauranter - a.Restauranter)[0];
        const highestAccommodation = [...sortedData].sort((a, b) => b.Overnatning - a.Overnatning)[0];
        
        // Calculate average percentages
        const avgPackagePct = sortedData.reduce((sum, item) => sum + item.Package_Pct, 0) / sortedData.length;
        const avgRestaurantPct = sortedData.reduce((sum, item) => sum + item.Restaurant_Pct, 0) / sortedData.length;
        const avgAccommodationPct = sortedData.reduce((sum, item) => sum + item.Accommodation_Pct, 0) / sortedData.length;
        
        return {{
          highestSpender,
          lowestSpender,
          highestPackage,
          highestRestaurant,
          highestAccommodation,
          avgPackagePct,
          avgRestaurantPct,
          avgAccommodationPct
        }};
      }};
      
      const insights = calculateInsights();
      
      // Determine if season data is available
      const hasSeasonalData = datasets.season && datasets.season.data && datasets.season.data.length > 0;
      
      // Render seasonal section
      const renderSeasonalSection = () => {{
        if (!hasSeasonalData) return null;
        
        return (
          <div className="chart-container">
            <div className="chart-title">Travel Seasons Preferences by Group</div>
            <div className="chart-subtitle">When different socioeconomic groups prefer to travel</div>
            
            <ResponsiveContainer width="100%" height={400}>
              <AreaChart data={{datasets.season.data}} margin={{{{top: 20, right: 30, left: 20, bottom: 10}}}}>
                <CartesianGrid strokeDasharray="3 3" />
                <XAxis dataKey="season" />
                <YAxis domain={[0, 50]} unit="%" />
                <Tooltip content={{<CustomTooltip />}} />
                <Legend />
                <Area type="monotone" dataKey="HighIncome" stackId="1" stroke="#8884d8" fill="#8884d8" name="High Income" />
                <Area type="monotone" dataKey="MediumIncome" stackId="2" stroke="#82ca9d" fill="#82ca9d" name="Medium Income" />
                <Area type="monotone" dataKey="BasicIncome" stackId="3" stroke="#ffc658" fill="#ffc658" name="Basic Income" />
                <Area type="monotone" dataKey="Student" stackId="4" stroke="#ff8042" fill="#ff8042" name="Student" />
                <Area type="monotone" dataKey="Pensioner" stackId="5" stroke="#0088fe" fill="#0088fe" name="Pensioner" />
              </AreaChart>
            </ResponsiveContainer>
            
            <div className="seasonal-grid">
              {{datasets.season.data.map((season, index) => (
                <div className="seasonal-card" key={{season.season}}>
                  <h3>{{season.season}}</h3>
                  
                  {{/* High Income */}}
                  <div>
                    <div className="seasonal-value">
                      <span>High Income:</span>
                      <span>{{season.HighIncome}}%</span>
                    </div>
                    <div className="seasonal-value-bar">
                      <div 
                        className="seasonal-value-fill" 
                        style={{{{
                          width: `${{season.HighIncome * 2}}%`, 
                          backgroundColor: '#8884d8'
                        }}}}
                      />
                    </div>
                  </div>
                  
                  {{/* Medium Income */}}
                  <div>
                    <div className="seasonal-value">
                      <span>Medium Income:</span>
                      <span>{{season.MediumIncome}}%</span>
                    </div>
                    <div className="seasonal-value-bar">
                      <div 
                        className="seasonal-value-fill" 
                        style={{{{
                          width: `${{season.MediumIncome * 2}}%`, 
                          backgroundColor: '#82ca9d'
                        }}}}
                      />
                    </div>
                  </div>
                  
                  {{/* Basic Income */}}
                  <div>
                    <div className="seasonal-value">
                      <span>Basic Income:</span>
                      <span>{{season.BasicIncome}}%</span>
                    </div>
                    <div className="seasonal-value-bar">
                      <div 
                        className="seasonal-value-fill" 
                        style={{{{
                          width: `${{season.BasicIncome * 2}}%`, 
                          backgroundColor: '#ffc658'
                        }}}}
                      />
                    </div>
                  </div>
                  
                  {{/* Student */}}
                  <div>
                    <div className="seasonal-value">
                      <span>Student:</span>
                      <span>{{season.Student}}%</span>
                    </div>
                    <div className="seasonal-value-bar">
                      <div 
                        className="seasonal-value-fill" 
                        style={{{{
                          width: `${{season.Student * 2}}%`, 
                          backgroundColor: '#ff8042'
                        }}}}
                      />
                    </div>
                  </div>
                  
                  {{/* Pensioner */}}
                  <div>
                    <div className="seasonal-value">
                      <span>Pensioner:</span>
                      <span>{{season.Pensioner}}%</span>
                    </div>
                    <div className="seasonal-value-bar">
                      <div 
                        className="seasonal-value-fill" 
                        style={{{{
                          width: `${{season.Pensioner * 2}}%`, 
                          backgroundColor: '#0088fe'
                        }}}}
                      />
                    </div>
                  </div>
                </div>
              ))}}
            </div>
          </div>
        );
      }};
      
      // Render the appropriate chart based on the view
      const renderChart = () => {{
        if (!sortedData || sortedData.length === 0) return <div>No data available</div>;
        
        switch (chartView) {{
          case 'radar':
            return (
              <ResponsiveContainer width="100%" height={500}>
                <RadarChart outerRadius={{180}} data={{sortedData}}>
                  <PolarGrid stroke="#e5e5e5" />
                  <PolarAngleAxis dataKey="translated" tick={{{{ fontSize: 12, fill: '#666' }}}} />
                  <PolarRadiusAxis tick={{false}} axisLine={{false}} />
                  
                  <Radar 
                    name="Travel Packages" 
                    dataKey="Rejsepakker" 
                    stroke={{colors.Rejsepakker}} 
                    fill={{colors.Rejsepakker}} 
                    fillOpacity={{activeCategory === null || activeCategory === "Rejsepakker" ? 0.6 : 0.2}}
                    activeDot={{{{ r: 6 }}}}
                  />
                  <Radar 
                    name="Restaurants" 
                    dataKey="Restauranter" 
                    stroke={{colors.Restauranter}} 
                    fill={{colors.Restauranter}} 
                    fillOpacity={{activeCategory === null || activeCategory === "Restauranter" ? 0.6 : 0.2}}
                    activeDot={{{{ r: 6 }}}}
                  />
                  <Radar 
                    name="Accommodation" 
                    dataKey="Overnatning" 
                    stroke={{colors.Overnatning}} 
                    fill={{colors.Overnatning}} 
                    fillOpacity={{activeCategory === null || activeCategory === "Overnatning" ? 0.6 : 0.2}}
                    activeDot={{{{ r: 6 }}}}
                  />
                  
                  <Tooltip content={{<CustomTooltip />}} />
                  <Legend 
                    onClick={{(entry) => handleLegendClick(entry)}}
                    formatter={{(value, entry) => {{
                      // Highlight active category in legend
                      const isActive = activeCategory === entry.dataKey || !activeCategory;
                      return <span style={{{{color: isActive ? entry.color : '#999', fontWeight: isActive ? 'bold' : 'normal'}}}}>{{value}}</span>;
                    }}}}
                  />
                </RadarChart>
              </ResponsiveContainer>
            );
          
          case 'bar':
            return (
              <ResponsiveContainer width="100%" height={500}>
                <BarChart data={{sortedData}} margin={{{{top: 20, right: 30, left: 20, bottom: 70}}}}>
                  <CartesianGrid strokeDasharray="3 3" />
                  <XAxis 
                    dataKey="translated" 
                    tick={{{{ fontSize: 12 }}}} 
                    angle={-45}
                    textAnchor="end"
                    height={70}
                  />
                  <YAxis />
                  <Tooltip content={{<CustomTooltip />}} />
                  <Legend onClick={{(entry) => handleLegendClick(entry)}} />
                  <Bar 
                    name="Travel Packages" 
                    dataKey="Rejsepakker" 
                    fill={{colors.Rejsepakker}} 
                    opacity={{activeCategory === null || activeCategory === "Rejsepakker" ? 1 : 0.3}}
                    animationDuration={1500}
                  />
                  <Bar 
                    name="Restaurants" 
                    dataKey="Restauranter" 
                    fill={{colors.Restauranter}} 
                    opacity={{activeCategory === null || activeCategory === "Restauranter" ? 1 : 0.3}}
                    animationDuration={1500}
                  />
                  <Bar 
                    name="Accommodation" 
                    dataKey="Overnatning" 
                    fill={{colors.Overnatning}} 
                    opacity={{activeCategory === null || activeCategory === "Overnatning" ? 1 : 0.3}}
                    animationDuration={1500}
                  />
                </BarChart>
              </ResponsiveContainer>
            );
          
          case 'pie':
            // Focus on total spending distribution for pie chart
            const pieData = sortedData.map(item => ({{
              name: item.group,
              translated: item.translated,
              value: item.Total
            }}));
            
            return (
              <div style={{{{display: 'flex', flexDirection: 'column', alignItems: 'center'}}}}>
                <div style={{{{marginBottom: '20px', color: '#7f8c8d', fontSize: '15px'}}}}>
                  Click on segments to see details
                </div>
                <ResponsiveContainer width="100%" height={400}>
                  <PieChart>
                    <Pie
                      activeIndex={{activePieIndex}}
                      activeShape={{renderActiveShape}}
                      data={{pieData}}
                      cx="50%"
                      cy="50%"
                      innerRadius={{60}}
                      outerRadius={{120}}
                      dataKey="value"
                      onMouseEnter={{handlePieEnter}}
                      animationDuration={1000}
                    >
                      {{pieData.map((entry, index) => (
                        <Cell key={{`cell-${{index}}`}} fill={{COLORS[index % COLORS.length]}} />
                      ))}}
                    </Pie>
                    <Tooltip />
                  </PieChart>
                </ResponsiveContainer>
                
                <div style={{{{marginTop: '20px', textAlign: 'center', maxWidth: '600px'}}}}>
                  <div style={{{{fontWeight: 'bold', fontSize: '18px', color: '#2c3e50', marginBottom: '10px'}}}}>
                    {{pieData[activePieIndex]?.translated}} - {{pieData[activePieIndex]?.value.toLocaleString()}} DKK
                  </div>
                  <div style={{{{color: '#7f8c8d', fontSize: '15px'}}}}>
                    This represents {{(pieData[activePieIndex]?.value / pieData.reduce((sum, item) => sum + item.value, 0) * 100).toFixed(1)}}% 
                    of total travel spending across all groups.
                  </div>
                </div>
              </div>
            );
          
          case 'composed':
            // Create data for the composed chart
            // Focusing on percentage distribution
            return (
              <ResponsiveContainer width="100%" height={500}>
                <ComposedChart 
                  data={{sortedData}}
                  margin={{{{top: 20, right: 20, bottom: 70, left: 20}}}}
                >
                  <CartesianGrid stroke="#f5f5f5" />
                  <XAxis 
                    dataKey="translated" 
                    scale="band"
                    tick={{{{ fontSize: 12 }}}} 
                    angle={-45}
                    textAnchor="end"
                    height={70}
                  />
                  <YAxis yAxisId="left" orientation="left" stroke="#8884d8" />
                  <YAxis yAxisId="right" orientation="right" stroke="#82ca9d" />
                  <Tooltip content={{<CustomTooltip />}} />
                  <Legend />
                  <Bar 
                    yAxisId="left" 
                    dataKey="Total" 
                    barSize={{40}} 
                    fill={colors.Total}
                    name="Total Spending"
                  />
                  <Line 
                    yAxisId="right" 
                    type="monotone" 
                    dataKey="Restaurant_Pct" 
                    stroke="#ff7300" 
                    name="Restaurant %" 
                    activeDot={{<CustomizedActiveDot />}}
                  />
                  <Line 
                    yAxisId="right" 
                    type="monotone" 
                    dataKey="Package_Pct" 
                    stroke="#387908" 
                    name="Travel Package %" 
                    activeDot={{<CustomizedActiveDot />}}
                  />
                  <Line 
                    yAxisId="right" 
                    type="monotone" 
                    dataKey="Accommodation_Pct" 
                    stroke="#38abc8" 
                    name="Accommodation %" 
                    activeDot={{<CustomizedActiveDot />}}
                  />
                </ComposedChart>
              </ResponsiveContainer>
            );
          
          default:
            return null;
        }}
      }};
      
      return (
        <div className="container">
          <div className="header">
            <h1>Danish Travel Budget Analysis</h1>
            <p>
              How different socioeconomic groups in Denmark budget for travel, showing spending patterns
              on packages, restaurants, and accommodation across various demographics.
            </p>
          </div>
          
          {{/* Dataset tabs */}}
          <div className="tabs">
            {{availableDatasets.map(key => (
              <button 
                key={{key}}
                className={{`tab-button ${{activeDataset === key ? 'active' : ''}}`}}
                onClick={{() => setActiveDataset(key)}}
              >
                {{displayNames[key]}}
              </button>
            ))}}
            {{hasSeasonalData && (
              <button 
                className={{`tab-button ${{activeDataset === 'season' ? 'active' : ''}}`}}
                onClick={{() => {{
                  setActiveDataset('season');
                  setChartView('radar'); // Reset to radar when switching to seasonal
                }}}}
              >
                {{displayNames['season']}}
              </button>
            )}}
          </div>
          
          {{activeDataset !== 'season' ? (
            <div className="chart-container">
              <div className="chart-title">{{datasets[activeDataset]?.title || 'Travel Budget Analysis'}}</div>
              
              {{/* Chart view selector */}}
              <div className="tabs">
                {{viewOptions.map(option => (
                  <button 
                    key={{option.id}}
                    className={{`tab-button ${{chartView === option.id ? 'active' : ''}}`}}
                    onClick={{() => setChartView(option.id)}}
                  >
                    {{option.name}}
                  </button>
                ))}}
              </div>
              
              {{/* Controls row with sorting */}}
              <div className="controls-row">
                <div className="chart-subtitle">Travel expenses in DKK</div>
                
                <div className="sort-control">
                  <label htmlFor="sort-select">Sort by:</label>
                  <select 
                    id="sort-select"
                    value={{sortBy}}
                    onChange={{(e) => setSortBy(e.target.value)}}
                  >
                    <option value="default">Default Order</option>
                    <option value="total">Total Spending</option>
                    <option value="Rejsepakker">Travel Packages</option>
                    <option value="Restauranter">Restaurants</option>
                    <option value="Overnatning">Accommodation</option>
                  </select>
                </div>
              </div>
              
              {{renderChart()}}
              
              <div className="hint">
                <div><strong>Interactive features:</strong></div>
                <div>• Click on legend items to highlight specific categories</div>
                <div>• Hover over chart elements to see detailed values</div>
                <div>• Use the sort dropdown to reorder the data</div>
                <div>• Try different chart types using the tabs above</div>
              </div>
              
              {{insights && (
                <div className="insights">
                  <h3>Key Insights</h3>
                  <ul>
                    <li>
                      <strong>{{insights.highestSpender.translated}}</strong> spend the most on travel overall 
                      ({{insights.highestSpender.Total.toLocaleString()}} DKK), while 
                      <strong> {{insights.lowestSpender.translated}}</strong> spend the least 
                      ({{insights.lowestSpender.Total.toLocaleString()}} DKK).
                    </li>
                    <li>
                      <strong>Restaurant spending</strong> accounts for the largest portion of travel budgets 
                      ({{insights.avgRestaurantPct.toFixed(1)}}% on average).
                    </li>
                    <li>
                      <strong>{{insights.highestPackage.translated}}</strong> invest the most in travel packages 
                      ({{insights.highestPackage.Rejsepakker.toLocaleString()}} DKK).
                    </li>
                    <li>
                      <strong>{{insights.highestRestaurant.translated}}</strong> spend the most on restaurants 
                      ({{insights.highestRestaurant.Restauranter.toLocaleString()}} DKK).
                    </li>
                    <li>
                      <strong>{{insights.highestAccommodation.translated}}</strong> allocate the most to accommodation 
                      ({{insights.highestAccommodation.Overnatning.toLocaleString()}} DKK).
                    </li>
                  </ul>
                </div>
              )}}
            </div>
          ) : (
            renderSeasonalSection()
          )}}
        </div>
      );
    }};
    
    ReactDOM.render(<App />, document.getElementById('root'));
  </script>
</body>
</html>
'''

# Ensure the output directory exists
os.makedirs('../assets', exist_ok=True)

# Save the HTML file
output_path = '../assets/danish_travel_interactive.html'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(html_template)

print(f"Enhanced interactive visualization created and saved to {output_path}")


SyntaxError: f-string: expecting a valid expression after '{' (3947010539.py, line 378)